# <a> Data Science in Projects - Project 2: Recommendation System</a>

## Let's Data - Data Science Journey

*By Reginaldo Jr - <a>reginaldojr@yahoo.com</a>*

## Imports and Configurations

In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

from matplotlib        import pyplot as plt
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

sns.set_style("darkgrid")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

warnings.filterwarnings("ignore")

## Data

In [52]:
df_movie = pd.read_csv("movies.csv", sep=",")
df_movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [53]:
df_movie = df_movie.drop(["genres"], axis=1)
df_movie.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [54]:
df_movie.shape

(9742, 2)

In [55]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
dtypes: int64(1), object(1)
memory usage: 152.3+ KB


In [56]:
df_rating = pd.read_csv("ratings.csv", sep=',')
df_rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.00,964982703
1,1,3,4.00,964981247
2,1,6,4.00,964982224
3,1,47,5.00,964983815
4,1,50,5.00,964982931


In [57]:
df_rating = df_rating.drop(["timestamp"], axis=1)
df_rating.head()

,userId,movieId,rating
0,1,1,4.00
1,1,3,4.00
2,1,6,4.00
3,1,47,5.00
4,1,50,5.00


In [58]:
df_rating.shape

(100836, 3)

In [59]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [60]:
df_count_rating = df_rating.groupby("movieId")["rating"].count().reset_index()
df_count_rating.head()

,movieId,rating
0,1,215
1,2,110
2,3,52
3,4,7
4,5,49


In [61]:
df_count_rating.columns = ["movieId", 'countRating']
df_count_rating.head()

,movieId,countRating
0,1,215
1,2,110
2,3,52
3,4,7
4,5,49


In [62]:
df_count_rating.shape

(9724, 2)

In [63]:
df_count_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9724 entries, 0 to 9723
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   movieId      9724 non-null   int64
 1   countRating  9724 non-null   int64
dtypes: int64(2)
memory usage: 152.1 KB


In [64]:
df_count_rating['countRating'].describe()

count   9724.00
mean      10.37
std       22.40
min        1.00
25%        1.00
50%        3.00
75%        9.00
max      329.00
Name: countRating, dtype: float64

In [65]:
len(df_count_rating.loc[df_count_rating["countRating"]>=50])

450

In [66]:
df = pd.merge(df_rating,df_count_rating.loc[df_count_rating["countRating"]>=50], on='movieId').drop(["countRating"], axis=1)
df.head()

,userId,movieId,rating
0,1,1,4.00
1,5,1,4.00
2,7,1,4.50
3,15,1,2.50
4,17,1,4.50


In [67]:
df.shape

(41360, 3)

In [68]:
df.isnull().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [69]:
movie_features_df = df.pivot_table(index='movieId',columns='userId',values='rating').fillna(0)
movie_features_df.tail()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
109374,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.00,2.00,0.00,0.00,4.00,0.00,0.00,3.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00,3.50,0.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,0.00,4.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,4.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,0.00,

In [70]:
movie_features_df.shape

(450, 606)

In [71]:
movie_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 450 entries, 1 to 122904
Columns: 606 entries, 1 to 610
dtypes: float64(606)
memory usage: 2.1 MB


## Implementation

In [72]:
movie_features_df_matrix = csr_matrix(movie_features_df) 

model_knn = NearestNeighbors(metric='cosine')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(metric='cosine')

In [73]:
def getTitle(movieId):
    return df_movie.loc[df_movie["movieId"]==movieId].values[0][1]

In [86]:
query_index = np.random.choice(movie_features_df.shape[0])
distances, indices = model_knn.kneighbors(movie_features_df.loc[movie_features_df.index==movie_features_df.index[query_index]].values, n_neighbors = 7)
print(f'Movie: {getTitle(movie_features_df.index[query_index])} - index {query_index}')

Movie: Dark Knight Rises, The (2012) - index 441


In [87]:
for i in range(0, len(distances[0])):
    if i == 0:
        print(f'Recommendation to {movie_features_df.index[query_index]} - {getTitle(movie_features_df.index[query_index])}:\n')
    else:
        print(f'{i}: {getTitle(movie_features_df.index[indices[0][i]])}, with distance of {distances[0][i]}:')

Recommendation to 91529 - Dark Knight Rises, The (2012):

1: Dark Knight, The (2008), with distance of 0.33394556662760566:
2: Inception (2010), with distance of 0.3824955344787052:
3: Avengers, The (2012), with distance of 0.38764447366883437:
4: Interstellar (2014), with distance of 0.41712086419487826:
5: Django Unchained (2012), with distance of 0.4194602836094039:
6: Batman Begins (2005), with distance of 0.4447542953971322:
